# [Shortest Path in a Grid with Obstacles Elimination](https://leetcode.com/problems/shortest-path-in-a-grid-with-obstacles-elimination/)
- **Given**: A grid `M`x`N`
    + 0: empty cell
    + 1: obstacle
- **Return** the **minimum number of steps** to walk from the upper left corner (0, 0) to the lower right corner (M-1, N-1)
    + Can eliminate at most `K` obstacles
    + If it is not possible return -1
    
#### Constraints

- $1 \leq M, N \leq 40$
- $1 \leq K \leq M*N$

#### Example 1
- $K = 1$
<img src="./assets/1.jpg" width="150"/>

```
Output: 6
Explanation: (0,0) -> (0,1) -> (0,2) -> (1,2) -> (2,2) -> (3,2) -> (4,2)
```

#### Example 2
- $K = 1$
<img src="./assets/2.jpg" width="150"/>

```
Output: -1
Explanation: We need to eliminate at least two obstacles to find such a walk.
```

## Solution - DP - $O(M*N*K)$

```C++
class Solution {
public:
    int shortestPath(vector<vector<int>> &A, int K) {
        int N = A.size();
        int M = A[0].size();

        // {x, y, k}
        queue<tuple<int,int,int>> Q;
        Q.push( {0,0, A[0][0]} );
        
        // dp[x][y][k] = distance at (x,y,k)
        vector<vector<vector<int>>> dp(N, vector<vector<int>>(M, vector<int>(K+1, INT_MAX)));
        dp[0][0][A[0][0]] = 0;

        // BFS DP; no early stopping
        while(!Q.empty()) {
            auto [x, y, k] = Q.front();
            Q.pop();

            for(auto [x_, y_]: vector<pair<int,int>>{{0,1}, {1,0}, {0,-1}, {-1, 0}}) {
                int x_next = x + x_;
                int y_next = y + y_;

                if(0 <= x_next && x_next < N &&
                        0 <= y_next && y_next < M) {
                    int k_next = (A[x_next][y_next] == 1) ? k + 1 : k;

                    // Note: Condition to update DP help prune the search tree
                    if(k_next <= K && dp[x_next][y_next][k_next] > dp[x][y][k] + 1) {
                        dp[x_next][y_next][k_next] = dp[x][y][k] + 1;
                        Q.push( {x_next, y_next, k_next} );
                    }
                }
            }
        }

        // Find the shortest path
        int ans = INT_MAX;
        for (int k=0; k<=K; ++k) {
            ans = min(ans, dp[N-1][M-1][k]);
        }
        return ans == INT_MAX ? -1: ans;
    }
};
```

## Solution - DP - $O(M*N)$
- **Ideas**
    + Reduce dp dimension to 2D
    + Use BFS travel by dist to keeptrack of dist (increase +1 by batch)

```Cpp
class Solution {
public:
    int shortestPath(vector<vector<int>> &A, int K) {
        int N = A.size();
        int M = A[0].size();

        // dp[x][y] = opt(k) at x,y
        vector<vector<int>> dp(N, vector<int>(M, INT_MAX));
        dp[0][0] = A[0][0];

        // {x, y}
        queue<pair<int,int>> Q;
        Q.push( {0,0} );

        // BFS: Keep track of dist
        int dist = -1;
        while(!Q.empty()) {
            int batch_size = Q.size();
            dist += 1;

            for(int _=0; _<batch_size; ++_) {
                auto [x, y] = Q.front();
                Q.pop();

                if(x == N-1 && y == M-1) return dist;

                for(auto [x_, y_]: vector<pair<int,int>>{{0,1},{1,0},{0,-1},{-1,0}}) {
                    int x_next = x + x_;
                    int y_next = y + y_;

                    if(0 <= x_next && x_next < N &&
                            0 <= y_next && y_next < M) {
                        int k_next = (A[x_next][y_next] == 1) ? dp[x][y] + 1 : dp[x][y];

                        // Relax Condition
                        if(k_next <= K && dp[x_next][y_next] > k_next) {
                            dp[x_next][y_next] = k_next;
                            Q.push( {x_next, y_next} );
                        } 
                    }
                }
            }
        }
        return -1;
    }
};
```